# Pytorch_llm_intro_ja.ipynb

こちらは日本語向けのLLM(Large Language Model)のチュートリアルです。<br>
ここでは、Microsoftによって学習された言語モデルを使います。<br>
これらのモデルは、[The MIT license](https://opensource.org/license/mit/)で提供されています。

このチュートリアルは、Google Colaboratoryで、GPUを使って実行されることを想定しています。

This is the tutorial for LLM(Large Language Model) for Japanese.<br>
We use the language model trained by Microsoft.<br>
These models are under [The MIT license](https://opensource.org/license/mit/).

I assume that this tutorial is used on Google Colaboratory with GPU.

-----

https://huggingface.co/microsoft/Phi-4-mini-instruct<br>
https://huggingface.co/microsoft/Phi-4-mini-flash-reasoning


これらが、今回使用したモデルのドキュメントとなっております。

These are the pages of documents for the models that we use this time.

## ライブラリをpip installする

ここでは、必要なライブラリをpipを用いてインストールします。

On this part we will pip install libraries that we need.

In [ ]:
! pip install transformers==4.50.0 accelerate

## ライブラリをインポートする

ここでは、必要なライブラリをインポートします。

- PyTorch
- transformers

PyTorchはGoogle Colaboratoryにはデフォルトでインストールされています。<br>
[transformers](https://github.com/huggingface/transformers)は、[HuggingFace社](https://huggingface.co/)が開発を行っている機械学習向けのライブラリです。

もし現時点でGPUが使えない状態の場合は、`False`が出力されます。<br>
もし`False`が出力されたら、GPUを使えるよう変更してください。

Here we import libries that we need.

- PyTorch
- transformers

Pytorch is pre-installed on Google Colaboratory.<br>
[transformers](https://github.com/huggingface/transformers) is library for machine learning that [HuggingFace, Inc.](https://huggingface.co/) develops.

If you can't use GPU now, this notebook outputs `False`.<br>
If this cell outputs `False`, you should change setting to use GPU.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print(torch.cuda.is_available())

## 会話の続きを予測する

まずは、会話の続きを予測するモデルを試してみましょう。

First, we use the model that predicts continuation of dialogue.

### TokenizerとModelを定義する

ファイルを読み込んで、`tokenizer`と`model`を定義します。<br>
少々時間が掛かります。

Read files and define `tokenizer` and `model`.<br>
It'll take a little bit time.

In [ ]:
model_path = "microsoft/Phi-4-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
    cache_dir="./instruct",
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

### Promptを定義する
`prompt`を定義します。<br>
ここで`prompt`とは、言語モデルに渡す会話のことを表します。<br>
**ここでは内容を自由に変更することができます。**

Define `prompt`.<br>
`prompt`, that we use for now, means conversation that we pass to language model.<br>
**Here you can change codes freely.**

In [ ]:
prompt = "東京科学大学とはどのような大学ですか。"

### 返答を予測する

ここで、先ほどのパートで定義した`prompt`を処理します。<br>
その後、返答を予測します。<br>
少々時間が掛かります。

Here `prompt` that you defined last part is processed.<br>
Then the model predicts reply.<br>
It'll take a little bit time.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": True,
}

output = pipe(messages, **generation_args)
result = output[0]['generated_text']
del output
torch.cuda.empty_cache()

print(result)

## 複雑な推論をする

次に、複雑な推論をするモデルを試してみましょう。<br>
しかし、このモデルは残念ながら英語にしか対応していません。<br>
そこで、先ほどの推論モデルに翻訳をしてもらって、その文章をモデルに入力しましょう。<br>
**以降、GPUメモリがあふれる可能性があります。その場合は「セッションを再起動」し、上の「TokenizerとModelを定義する」のセルだけ実行し、以下のセルを実行してください。**

Next, let's try the model that does complex inference.<br>
However, it's sorry that this model only supports English.<br>
Therefore, we use the former model with translating the problem, and then input it to this model.<br>
**From now on, GPU memory might overflow. If so, please restart the session and run only former "TokenizerとModelを定義する" cell, then run all following cells.**

### TokenizerとModelを定義する

ファイルを読み込んで、`tokenizer`と`model`を定義します。<br>
少々時間が掛かります。

Read files and define `tokenizer` and `model`.<br>
It'll take a little bit time.

In [ ]:
model_id = "microsoft/Phi-4-mini-reasoning"
inf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
    cache_dir="./reasoning",
)
inf_tokenizer = AutoTokenizer.from_pretrained(model_id)

### Problemを定義する
`problem`を定義します。<br>
ここで`problem`とは、言語モデルに渡す問題のことを表します。<br>
**ここでは内容を自由に変更することができます。**

Define `problem`.<br>
`problem`, that we use for now, means problem that we pass to language model.<br>
**Here you can change codes freely.**

In [ ]:
problem = "x^2-3x+6=4を解いてください。"

### 解答を推論する

ここで、先ほど出された問題を、いったん英語に翻訳します。<br>
その後、推論モデルに入力して、問題の回答を出力させます。<br>
10分ほど時間が掛かります。<br>

Here the problem is translated into English.<br>
Then we input it to the inference model and it will output the answer for the problem.<br>
It'll take about 10 minutes.

In [ ]:
translate_messages = [
    {"role": "system", "content": "You are a good translator."},
    {"role": "user", "content": f"How can I say 「{problem}」 in English? Answer only the translated sentence."},
]

output = pipe(translate_messages, **generation_args)
english_problem = output[0]['generated_text']
del output
torch.cuda.empty_cache()
print(f"翻訳された問題: {english_problem}")

problem_messages = [{
    "role": "user",
    "content": english_problem,
}]
inputs = inf_tokenizer.apply_chat_template(
    problem_messages,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt",
)
outputs = inf_model.generate(
    **inputs.to(inf_model.device),
    max_new_tokens=32768,
    temperature=0.6,
    top_p=0.95,
    do_sample=True,
)
outputs = inf_tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])
english_answer = outputs[0]
del inputs, outputs
torch.cuda.empty_cache()

jp_translate = [
    {"role": "system", "content": "You are a good translator."},
    {"role": "user", "content": f"「{english_answer}」を日本語に翻訳してください。"}
]

output2 = pipe(jp_translate, **generation_args)
japanese_answer = output2[0]['generated_text']
del output2
torch.cuda.empty_cache()
print(f"解答: {japanese_answer}")